# Draft and test

In [3]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import pandas as pd

# use interactive plotting 
from ipywidgets import interact, interactive, fixed, interact_manual

## 1. Data

### 1.1 Data from HATPRO

Using the network data causes a much longer run time

### 1.1 Select days available in both setups
Data from local folders

In [4]:
# find all directories which have the same name in the folder in exp_path/icon_burga_v1 and exp_path/icon_burga_v0 and store the first 8 digits of the names in a list 
#exp_path = '/data/inscape/icon/experiments/nyalesund/testbed/'
exp_path='../../data/'
exp_name_v1 = 'exp_data_v1/'
exp_name_v0 = 'exp_data_v0/'

# find all directories which have the same name in the folder in exp_path/icon_burga_v1 and exp_path/icon_burga_v0 and store the first 8 digits of the names in a list
exp_dirs_v1 = glob.glob(exp_path+exp_name_v1+'*')
exp_dirs_v0 = glob.glob(exp_path+exp_name_v0+'*')
print(exp_dirs_v1)
# sort the lists
exp_dirs_v1.sort()
exp_dirs_v0.sort()

# create a list with the first 8 digits of the names which come after the last / in the directory names
exp_names_v1 = [exp_dir.split('/')[-1][19:27] for exp_dir in exp_dirs_v1]
exp_names_v0 = [exp_dir.split('/')[-1][19:27] for exp_dir in exp_dirs_v0]
print(exp_names_v1)

# compare the lists and find the common elements
exp_dates = list(set(exp_names_v1) & set(exp_names_v0))

print(len(exp_dates))

# only select the elements which are between 20210701 and 202112031 to decrease data size and to 
#exp_dates = [exp_date for exp_date in exp_dates if int(exp_date) >= 20210801 and int(exp_date) <= 20210805]
print(len(exp_dates))
print(exp_dates)
# delete 20210905 because the Meteogram doesn't exist for this date
#exp_dates.remove('20210905')

['../../data/exp_data_v1/METEOGRAM_patch001_20210802_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211026_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20210815_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211031_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20210829_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211001_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211019_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20210825_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211024_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211007_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211018_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211008_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20210804_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20210811_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch001_20211014_awipev.nc', '../../data/exp_data_v1/METEOGRAM_patch

Obs data from HATPRO

In [44]:
hatpro_path = '/data/obs/site/nya/nyhat/l2/2021/'
hatpro_files_lwp = [hatpro_path+'/'+exp_date[4:6]+'/'+exp_date[6:8]+'/sups_nya_mwr00_l2_clwvi_v00_'+exp_date+'0000*.nc' for exp_date in exp_dates]
print(hatpro_files_lwp)
# get full names of the files 
hatpro_files_lwp = [glob.glob(hatpro_file) for hatpro_file in hatpro_files_lwp]
print(hatpro_files_lwp)
# flatten the list, needs to be done otherwise the files cannot be read by mfdataset
hatpro_files_lwp = [item for sublist in hatpro_files_lwp for item in sublist]
print(hatpro_files_lwp)

hatpro_files_iwv = [hatpro_path+'/'+exp_date[4:6]+'/'+exp_date[6:8]+'/sups_nya_mwr00_l2_prw_v00_'+exp_date+'0000*.nc' for exp_date in exp_dates]
hatpro_files_iwv = [glob.glob(hatpro_file) for hatpro_file in hatpro_files_iwv]
hatpro_files_iwv = [item for sublist in hatpro_files_iwv for item in sublist]

hatpro_lwp = xr.open_mfdataset(hatpro_files_lwp, combine='by_coords', format='NETCDF4', parallel=True)
hatpro_iwv = xr.open_mfdataset(hatpro_files_iwv, combine='by_coords', format='NETCDF4', parallel=True)  #.sel(time=slice('2021-08-01T00:00:00','2021-08-05T23:59:59'))

['/data/obs/site/nya/nyhat/l2/2021//10/18/sups_nya_mwr00_l2_clwvi_v00_202110180000*.nc', '/data/obs/site/nya/nyhat/l2/2021//08/22/sups_nya_mwr00_l2_clwvi_v00_202108220000*.nc', '/data/obs/site/nya/nyhat/l2/2021//10/14/sups_nya_mwr00_l2_clwvi_v00_202110140000*.nc', '/data/obs/site/nya/nyhat/l2/2021//08/08/sups_nya_mwr00_l2_clwvi_v00_202108080000*.nc', '/data/obs/site/nya/nyhat/l2/2021//08/23/sups_nya_mwr00_l2_clwvi_v00_202108230000*.nc', '/data/obs/site/nya/nyhat/l2/2021//08/19/sups_nya_mwr00_l2_clwvi_v00_202108190000*.nc', '/data/obs/site/nya/nyhat/l2/2021//10/23/sups_nya_mwr00_l2_clwvi_v00_202110230000*.nc', '/data/obs/site/nya/nyhat/l2/2021//10/27/sups_nya_mwr00_l2_clwvi_v00_202110270000*.nc', '/data/obs/site/nya/nyhat/l2/2021//08/03/sups_nya_mwr00_l2_clwvi_v00_202108030000*.nc', '/data/obs/site/nya/nyhat/l2/2021//10/10/sups_nya_mwr00_l2_clwvi_v00_202110100000*.nc', '/data/obs/site/nya/nyhat/l2/2021//10/29/sups_nya_mwr00_l2_clwvi_v00_202110290000*.nc', '/data/obs/site/nya/nyhat/l2/20

Simulation data

In [22]:
# open all the files which are in the common elements list exp_dates
exp_dir_names_v0 = [exp_path+exp_name_v0+'/METEOGRAM_patch001_'+exp_date+'_awipev.nc' for exp_date in exp_dates]
exp_dir_names_v1 = [exp_path+exp_name_v1+'/METEOGRAM_patch001_'+exp_date+'_awipev.nc' for exp_date in exp_dates]
print(exp_dir_names_v0)

ds_v1 = xr.open_mfdataset(exp_dir_names_v1, combine='by_coords', format='NETCDF4', parallel=True)
ds_v0 = xr.open_mfdataset(exp_dir_names_v0, combine='by_coords', format='NETCDF4', parallel=True)

['../../data/exp_data_v0//METEOGRAM_patch001_20211018_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20210822_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20211014_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20210808_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20210823_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20210819_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20211023_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20211027_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20210803_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20211010_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20211029_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20210801_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20210802_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20211025_awipev.nc', '../../data/exp_data_v0//METEOGRAM_patch001_20211020_awipev.nc', '../../data/exp_data_v0/

In [25]:
# remove the first three hours each day
ds_v1 = ds_v1.where(ds_v1.time.dt.hour >= 3, drop=True)
ds_v0 = ds_v0.where(ds_v0.time.dt.hour >= 3, drop=True)

# select only the first 100 layers of height and 99 layers of height_2 
ds_v1 = ds_v1.isel(height=slice(50,len(ds_v1.height)), height_2=slice(50,len(ds_v1.height_2)))
ds_v0 = ds_v0.isel(height=slice(50,len(ds_v0.height)), height_2=slice(50,len(ds_v0.height_2)))


In [45]:
print(hatpro_lwp.time.size)
hatpro_lwp_noflag = hatpro_lwp.where(np.isnan(hatpro_lwp.flag), drop=True)
hatpro_iwv_noflag = hatpro_iwv.where(np.isnan(hatpro_iwv.flag), drop=True)
print(hatpro_lwp_noflag.time.size, hatpro_iwv_noflag.time.size)

hatpro_lwp.flag

3736393
3406579 3406579


<xarray.DataArray 'flag' (time: 3736393)>
dask.array<concatenate, shape=(3736393,), dtype=float32, chunksize=(70311,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2021-08-01T00:00:22.999995904 ... 2021-10-...
Attributes:
    long_name:      quality control flags
    flag_masks:     [   1    2    4    8   16   32   64  128  256  512 1024]
    flag_meanings:  manual_filter_band_1 manual_filter_band2 manual_filter_ba...
    comment:        Flags indicate data that the user should only use with ca...

In [49]:
ds_v0.RAIN_GSP

<xarray.DataArray 'RAIN_GSP' (time: 504000)>
dask.array<where, shape=(504000,), dtype=float64, chunksize=(8400,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2021-08-01T03:00:00 ... 2021-10-31T23:59:51
Attributes:
    name:       RAIN_GSP
    long_name:  accumulated grid-scale surface rain
    unit:       kg/m2
    group_id:   3
    comment:    surface variable

### 2. make dataframe

In [10]:
# create a dataframe containing QNC, QNI, QNS, QNR, QNG, QNH
df_QN_v1 = pd.DataFrame({'QNC': ds_v1['QNC'].values.flatten(),
                        'QNI': ds_v1['QNI'].values.flatten(),
                        'QNS': ds_v1['QNS'].values.flatten(),
                        'QNR': ds_v1['QNR'].values.flatten(),
                        'QNG': ds_v1['QNG'].values.flatten(),
                        'QNH': ds_v1['QNH'].values.flatten(),
                        'T': ds_v1['T'].values.flatten()},
                        columns=['QNC', 'QNI', 'QNS', 'QNR', 'QNG', 'QNH', 'T']
                        )


In [11]:

df_QN_v0 = pd.DataFrame({'QNC': ds_v0['QNC'].values.flatten(),
                        'QNI': ds_v0['QNI'].values.flatten(),
                        'QNS': ds_v0['QNS'].values.flatten(),
                        'QNR': ds_v0['QNR'].values.flatten(),
                        'QNG': ds_v0['QNG'].values.flatten(),
                        'QNH': ds_v0['QNH'].values.flatten(),
                        'T': ds_v0['T'].values.flatten()},
                        columns=['QNC', 'QNI', 'QNS', 'QNR', 'QNG', 'QNH', 'T']
                        )

In [12]:
df_Q_v1 = pd.DataFrame({'QC': ds_v1['QC'].values.flatten(),
                        'QI': ds_v1['QI'].values.flatten(),
                        'QS': ds_v1['QS'].values.flatten(),
                        'QR': ds_v1['QR'].values.flatten(),
                        'QG': ds_v1['QG'].values.flatten(),
                        'QH': ds_v1['QH'].values.flatten(),
                        'T': ds_v1['T'].values.flatten()},
                        columns=['QC', 'QI', 'QS', 'QR', 'QG', 'QH', 'T']
                        )


In [13]:

df_Q_v0 = pd.DataFrame({'QC': ds_v0['QC'].values.flatten(),
                        'QI': ds_v0['QI'].values.flatten(),
                        'QS': ds_v0['QS'].values.flatten(),
                        'QR': ds_v0['QR'].values.flatten(),
                        'QG': ds_v0['QG'].values.flatten(),
                        'QH': ds_v0['QH'].values.flatten(),
                        'T': ds_v0['T'].values.flatten()},
                        columns=['QC', 'QI', 'QS', 'QR', 'QG', 'QH', 'T']
                        )

In [9]:
df_intvals_v1 = pd.DataFrame({'TQC': ds_v1['TQC'].values.flatten(),
                        'TQI': ds_v1['TQI'].values.flatten(),
                        'TQS': ds_v1['TQS'].values.flatten(),
                        'TQR': ds_v1['TQR'].values.flatten(),
                        'TQV': ds_v1['TQV'].values.flatten(),
                        'RAIN_GSP': ds_v1['RAIN_GSP'].values.flatten(),
                        'SNOW_GSP': ds_v1['SNOW_GSP'].values.flatten()},
                        columns=['TQC', 'TQI', 'TQS', 'TQR', 'TQV', 'RAIN_GSP', 'SNOW_GSP']
                        )

In [11]:
df_intvals_v0 = pd.DataFrame({'TQC': ds_v0['TQC'].values.flatten(),
                        'TQI': ds_v0['TQI'].values.flatten(),
                        'TQS': ds_v0['TQS'].values.flatten(),
                        'TQR': ds_v0['TQR'].values.flatten(),
                        'TQV': ds_v0['TQV'].values.flatten(),
                        'RAIN_GSP': ds_v0['RAIN_GSP'].values.flatten(),
                        'SNOW_GSP': ds_v0['SNOW_GSP'].values.flatten()},
                        columns=['TQC', 'TQI', 'TQS', 'TQR', 'TQV', 'RAIN_GSP', 'SNOW_GSP']
                        )



In [46]:
df_intvals_hatpro = pd.DataFrame({'LWP': hatpro_lwp_noflag['clwvi'].values.flatten(),
                        'IWV': hatpro_iwv_noflag['prw'].values.flatten()},
                        columns=['LWP', 'IWV']
                        )

In [47]:
display(df_intvals_hatpro.describe())

,LWP,IWV
count,3.035742e+06,3.035742e+06
mean,6.439846e-02,9.201725e+00
std,1.876736e-01,4.615121e+00
min,-1.621298e-02,2.453915e+00
25%,6.475820e-04,4.673852e+00
50%,1.247828e-02,9.702703e+00
75%,6.530567e-02,1.233816e+01
max,3.004606e+00,4.739595e+01


In [15]:
# only select values where there are clouds using sum of df_Q variables
df_QN_cloudy_v1 = df_QN_v1[(df_Q_v1['QC']+df_Q_v1['QI']+df_Q_v1['QS']+df_Q_v1['QR']+df_Q_v1['QG']+df_Q_v1['QH']) >= 1e-8]
df_Q_cloudy_v1 = df_Q_v1[(df_Q_v1['QC']+df_Q_v1['QI']+df_Q_v1['QS']+df_Q_v1['QR']+df_Q_v1['QG']+df_Q_v1['QH']) >= 1e-8]

df_QN_cloudy_v0 = df_QN_v0[(df_Q_v0['QC']+df_Q_v0['QI']+df_Q_v0['QS']+df_Q_v0['QR']+df_Q_v0['QG']+df_Q_v0['QH']) >= 1e-8]
df_Q_cloudy_v0 = df_Q_v0[(df_Q_v0['QC']+df_Q_v0['QI']+df_Q_v0['QS']+df_Q_v0['QR']+df_Q_v0['QG']+df_Q_v0['QH']) >= 1e-8]

In [16]:
#store the dataframe in a csv file
df_QN_cloudy_v1.to_csv('../../data/processed/df_QN_v1_AO_2021_cloudy.csv')
df_QN_cloudy_v0.to_csv('../../data/processed/df_QN_v0_AO_2021_cloudy.csv')

df_Q_cloudy_v1.to_csv('../../data/processed/df_Q_v1_AO_2021_cloudy.csv')
df_Q_cloudy_v0.to_csv('../../data/processed/df_Q_v0_AO_2021_cloudy.csv')

In [13]:
df_intvals_v0.to_csv('../../data/processed/df_intvals_precip_v0_AO_2021.csv')
df_intvals_v1.to_csv('../../data/processed/df_intvals_precip_v1_AO_2021.csv')

In [48]:
df_intvals_hatpro.to_csv('../../data/processed/df_intvals_hatpro_AO_2021.csv')